# RETO 1 NAYIBE BAYES
por:
***PAULINA BLANCO***

# 1. Se importan las Librerias Necesarias

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB

# 2. Se Importan los de Datos

In [2]:
#Se conecta Google Drive con Colab para poder acceder al archivo Excel.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Se lee el archivo Excel y se guarda en un DataFrame llamado solicitantes.
solicitantes = pd.read_excel("/content/drive/MyDrive/EAFIT/solicitantes_credito.xlsx")

In [7]:
#Muestra las primeras filas para verificar que los datos cargaron correctamente.
solicitantes.head()

,Unnamed: 0,Edad,Sexo,Estado civil,Hijos,Perscargo,Estrato,Nivel de estudios,Tipo de contrato,Ingresos,...,Plazo,Cuota (COP),ConoCliente,Garantia,RespPat,MoraCome,PreApr,Prob.Default (PD),Score,LGD
0,1,38,NR,Casado,1,0,3,Bachiller,Ninguno,5424576,...,48,82691.506204,Buenas Referencias,Sin Codeudor,Sin Patrimonio,Con Experiencia sin Mora,1.0,0.041659,582.478943,0.063702
1,2,51,NR,Separado,6,1,4,Técnico,Ninguno,1144064,...,36,22781.212598,Buenas Referencias,Sin Codeudor,Prop.sin Hipoteca,Con Experiencia sin Mora,1.0,0.234951,479.839966,0.465636
2,3,37,Mujer,Casado,2,0,3,Tecnólogo,Indefinido,2069853,...,36,46294.596244,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,Con Experiencia sin Mora,0.0,0.170330,417.653107,0.290071
3,4,29,NR,Separado,1,0,2,Tecnólogo,Ninguno,1893080,...,24,69564.950015,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,Con Experiencia sin Mora,1.0,0.201659,316.791107,0.278389
4,5,42,Mujer,Casado,3,4,2,Especializacion,Ninguno,3000367,...,24,112478.307883,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,Con Experiencia sin Mora,1.0,0.140929,441.809570,0.203359


# 3. Limpiamos los Datos

In [24]:
#Permite conocer cuántas filas y columnas tiene la base de datos original.
solicitantes.shape

(8378, 35)

In [25]:
## Elimino los registros donde PreApr está vacío
XD = solicitantes.dropna(subset=['PreApr'])

In [26]:
## Escojo solo las variables que voy a usar como independientes.
XD = XD[['Edad','Hijos','Perscargo','Estrato','Ingresos','Egresos','Monto (EAD)','Plazo','Cuota (COP)']]

In [27]:
XD

,Edad,Hijos,Perscargo,Estrato,Ingresos,Egresos,Monto (EAD),Plazo,Cuota (COP)
0,38,1,0,3,5424576,7585302,2.815030e+06,48,82691.506204
1,51,6,1,4,1144064,1640079,6.301439e+05,36,22781.212598
2,37,2,0,3,2069853,2831440,1.280540e+06,36,46294.596244
3,29,1,0,2,1893080,2482276,1.393414e+06,24,69564.950015
4,42,3,4,2,3000367,3630221,2.252986e+06,24,112478.307883
...,...,...,...,...,...,...,...,...,...
8371,48,0,0,3,4831392,3392105,3.366188e+06,36,121695.762156
8373,31,2,0,5,5891088,4292158,3.918572e+06,48,115108.056436
8374,38,0,0,3,3092079,3028098,2.677266e+06,36,96789.562091
8375,43,1,2,2,2542007,3513112,1.375112e+06,36,49713.608249


In [13]:
## Escojo la variable dependiente
yd = solicitantes['PreApr'].dropna()

# 5. Implementamos el modelo de Naive Bayes

In [16]:
### Convierto los datasets a arrays
XDArray = np.array(XD)
ydArray = np.array(yd)

In [17]:
### ENTRENAMOS EL MODELO (Fase Train)
modeloNB = GaussianNB()  ## Variable que aloja el modelo.
modeloNB.fit(XDArray, ydArray)

GaussianNB()

In [18]:
## Predecimos los yd (Etapa test)
ydPred = modeloNB.predict(XDArray)

# 6. Matriz de Confusion

In [19]:
from sklearn.metrics import confusion_matrix

In [29]:
cm = confusion_matrix(ydArray, ydPred)
cm

array([[2539,  420],
       [1041, 1842]])

# 7. Métricas de Evaluación del Modelo

In [32]:
## Extraigo los valores de la matriz de confusión
#VN (Verdaderos Negativos): 2539

VN = cm[0,0] ; FP = cm[0,1] ; FN = cm[1,0] ; VP = cm[1,1]

print(f'VN (Verdaderos Negativos):{VN}, FP (Falsos Positivos):{FP}, FN (Falsos Negativos):{FN}, VP (Verdaderos Positivos: {VP})')

VN (Verdaderos Negativos):2539, FP (Falsos Positivos):420, FN (Falsos Negativos):1041, VP (Verdaderos Positivos: 1842)


In [33]:
## Exactitud
exactitud = (VP + VN)/ cm.sum()

## Error
error = (FP +FN)/ cm.sum()

##Precisión
precisión = VP/(FP+VP)

##Sensibilidad
sensibilidad = VP/(VP+FN)

##Especificidad
especificidad = VN/(FP+VN)

In [23]:
## Imprimo los mensajes
print(f'La exactitud del modelo es de {exactitud:.2%}')
print(f'El error del modelo es de {error:.2%}')
print(f'La precisión del modelo es de {precisión:.2%}')
print(f'La sensibilidad del modelo es de {sensibilidad:.2%}')
print(f'La especificidad del modelo es de {especificidad:.2%}')

La exactitud del modelo es de 74.99%
El error del modelo es de 25.01%
La precisión del modelo es de 81.43%
La sensibilidad del modelo es de 63.89%
La especificidad del modelo es de 85.81%


#Análisis de las métricas con base a los resultados obtenidos:
Para este taller se volvió a entrenar el modelo Naive Bayes usando nuevas variables socioeconómicas como Edad, Hijos, Personas a Cargo, Estrato, Ingresos, Egresos, Monto, Plazo y Cuota. Luego, los resultados obtenidos se compararon con los del ejemplo trabajado en clase.

1. Exactitud: En el ejemplo visto en clase, la exactitud del modelo fue del 76.14%, mientras que en el modelo del taller fue del 74.99%, por lo que se presenta una leve disminución. Esto puede deberse a que, al incluir más variables financieras, el modelo tiene que evaluar una situación más cercana a la realidad, lo cual hace que la clasificación sea un poco más compleja y no siempre acierte en todos los casos.

2. Error: El error del modelo pasó de 23.86% en el ejemplo de clase a 25.01% en el modelo del taller, por lo que aumentó ligeramente. Este resultado está relacionado con la disminución de la exactitud. Al utilizar más información económica, el modelo se vuelve más exigente y puede equivocarse en aquellos casos donde los perfiles de los solicitantes no son tan claros.

3. Precisión: La precisión del modelo en clase fue del 81.01%, mientras que en el modelo del taller aumentó a 81.43%, lo que indica una mejora. Esto significa que cuando el modelo decide preaprobar un crédito, lo hace con mayor seguridad. La inclusión de variables como ingresos, egresos y cuota ayuda a que el modelo identifique mejor a los solicitantes que realmente tienen capacidad de pago, reduciendo así aprobaciones incorrectas.

4. Sensibilidad: La sensibilidad disminuyó de 67.46% en el modelo de clase a 63.89% en el modelo del taller. Esto indica que el modelo deja de identificar a algunos solicitantes que sí podrían ser preaprobados. Esto sucede porque, al tener en cuenta variables financieras, el modelo se vuelve más conservador y prefiere rechazar ciertos perfiles para disminuir el riesgo.

5. Especificidad: La especificidad aumentó de 84.59% a 85.81%, lo que representa una mejora. Esto quiere decir que el modelo es más efectivo identificando correctamente a los solicitantes que no deben ser preaprobados, lo cual es positivo desde el punto de vista del control del riesgo crediticio.

En conclusión, al incluir nuevas variables socioeconómicas en el modelo Naive Bayes, se evidencia que el modelo adopta un comportamiento más conservador. Aunque la exactitud y la sensibilidad presentan una leve disminución, la precisión y la especificidad mejoran, lo que indica que el modelo reduce el riesgo de aprobar créditos de manera incorrecta. Esto se debe a que, al agregar más variables financieras y socioeconómicas, el modelo se vuelve más estricto en la evaluación de los solicitantes, pero a la vez más seguro y confiable al momento de tomar decisiones de crédito.